In [2]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [4]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 0


# Check the drug

In [5]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization

In [7]:
optimizer_init_file_name = "optimizer/optimizer_init.json"

In [8]:
ax_client_init = BayesianOptimizer_Auto.initialize_ax()
ax_client_init.save_to_json_file(optimizer_init_file_name)

[INFO 02-10 16:09:54] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-10 16:09:54] ax.service.utils.instantiation: Due to non-specification, we will use the heuristic for selecting objective thresholds.
[INFO 02-10 16:09:54] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='Drug_MW', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='Drug_LogP', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='Drug_TPSA', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='Drug', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='SL_1', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='SL_2', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='SL_3', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='LL_1', parameter_t

# Generate recommendations

In [9]:
time_start = time.time()

# generate recomms
ax_client_init = AxClient.load_from_json_file(optimizer_init_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=0)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-10 16:09:58] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 02-10 16:09:58] ax.service.ax_client: Generated new trial 0 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 0.475107, 'SL_1': 0.592524, 'SL_2': 0.494479, 'SL_3': 0.313852, 'LL_1': 0.956092, 'LL_2': 0.083302, 'LL_3': 0.098763, 'P_1': 0.347466, 'P_2': 0.846915, 'P_3': 0.761508, 'S_1': 0.882807, 'S_2': 0.880169, 'S_3': 0.802465, 'Water': 0.851061} using model Sobol.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax

0 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,7.500000,354.0,3.91,67.0,0.098062,0.099312,0.101000,0.098062,0.097125,0.100562,0.10225,0.098937,0.104437,0.100438,0.256500,0.244000,0.254563,0.245000
std,4.760952,0.0,0.00,0.0,0.054100,0.060013,0.061061,0.059725,0.053875,0.057639,0.06276,0.057621,0.065002,0.061404,0.138986,0.125042,0.145372,0.149518
min,0.000000,354.0,3.91,67.0,0.007000,0.019000,0.003000,0.001000,0.003000,0.004000,0.00300,0.004000,0.006000,0.012000,0.021000,0.050000,0.031000,0.002000
25%,3.750000,354.0,3.91,67.0,0.053250,0.052500,0.061000,0.055250,0.046500,0.059500,0.05175,0.066250,0.046750,0.058750,0.153750,0.170000,0.152000,0.163500
50%,7.500000,354.0,3.91,67.0,0.108000,0.093500,0.094500,0.089500,0.102500,0.098500,0.11000,0.085000,0.110000,0.095500,0.260000,0.252500,0.245500,0.234000
75%,11.250000,354.0,3.91,67.0,0.139250,0.148500,0.142750,0.140500,0.129000,0.133000,0.15125,0.141000,0.162500,0.140500,0.351500,0.355000,0.375500,0.325000
max,15.000000,354.0,3.91,67.0,0.173000,0.196000,0.201000,0.217000,0.192000,0.219000,0.19800,0.215000,0.208000,0.245000,0.517000,0.484000,0.470000,0.600000


In [11]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,7.500000,0.354,0.391,0.067,0.500001,0.500999,0.500000,0.498016,0.500000,0.507812,0.500000,0.499998,0.500001,0.500244,0.500000,0.500488,0.500000,0.500000
std,4.760952,0.000,0.000,0.000,0.301481,0.296496,0.300534,0.298201,0.297979,0.301502,0.303306,0.296825,0.299873,0.294617,0.296462,0.293244,0.296058,0.298353
min,0.000000,0.354,0.391,0.067,0.026817,0.056699,0.018513,0.002008,0.011698,0.019458,0.009290,0.022020,0.023659,0.058300,0.055382,0.051906,0.041028,0.001178
25%,3.750000,0.354,0.391,0.067,0.259820,0.266574,0.245543,0.279797,0.271453,0.282965,0.258360,0.261866,0.248221,0.265395,0.260555,0.278861,0.263003,0.296029
50%,7.500000,0.354,0.391,0.067,0.490992,0.485835,0.497859,0.488928,0.517082,0.509079,0.496566,0.496061,0.508303,0.486588,0.519600,0.503314,0.502914,0.483384
75%,11.250000,0.354,0.391,0.067,0.743982,0.751436,0.750495,0.741004,0.740273,0.730021,0.738252,0.726504,0.738112,0.752640,0.715810,0.719898,0.730205,0.740136
max,15.000000,0.354,0.391,0.067,0.995107,0.975640,0.978429,0.948504,0.956092,0.993635,0.997516,0.954357,0.989036,0.969560,0.983573,0.938995,0.968647,0.961680


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000
mean,7.500000,354.0,3.91,67.0,34.321875,34.759375,35.350000,34.321875,33.993750,35.196875,35.787500,34.628125,36.553125,35.153125,256.500000,244.00000,254.562500,245.000000,-0.065625
std,4.760952,0.0,0.00,0.0,18.935170,21.004557,21.371219,20.903907,18.856253,20.173675,21.966137,20.167399,22.750527,21.491242,138.985851,125.04186,145.372152,149.517892,0.343375
min,0.000000,354.0,3.91,67.0,2.450000,6.650000,1.050000,0.350000,1.050000,1.400000,1.050000,1.400000,2.100000,4.200000,21.000000,50.00000,31.000000,2.000000,-0.700000
25%,3.750000,354.0,3.91,67.0,18.637500,18.375000,21.350000,19.337500,16.275000,20.825000,18.112500,23.187500,16.362500,20.562500,153.750000,170.00000,152.000000,163.500000,-0.350000
50%,7.500000,354.0,3.91,67.0,37.800000,32.725000,33.075000,31.325000,35.875000,34.475000,38.500000,29.750000,38.500000,33.425000,260.000000,252.50000,245.500000,234.000000,0.000000
75%,11.250000,354.0,3.91,67.0,48.737500,51.975000,49.962500,49.175000,45.150000,46.550000,52.937500,49.350000,56.875000,49.175000,351.500000,355.00000,375.500000,325.000000,0.087500
max,15.000000,354.0,3.91,67.0,60.550000,68.600000,70.350000,75.950000,67.200000,76.650000,69.300000,75.250000,72.800000,85.750000,517.000000,484.00000,470.000000,600.000000,0.350000


In [14]:
print("For 3 mm aspiration height, dead volume of 20 mL vials is around 2.5 mL") 

round(transfer[['Drug', 'SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000, 2)

For 3 mm aspiration height, dead volume of 20 mL vials is around 2.5 mL


Drug       0.55
SL_1       0.56
SL_2       0.57
SL_3       0.55
LL_1       0.54
LL_2       0.56
LL_3       0.57
P_1        0.55
P_2        0.58
P_3        0.56
S_1        4.10
S_2        3.90
S_3        4.07
Water      3.92
Solvent   -0.00
dtype: float64

In [6]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [16]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-10 16:11:07] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_0.json`.


# Generate new protocol

In [7]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [18]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [1]:
updated_script

NameError: name 'updated_script' is not defined

# Perform experiment

# Results

In [8]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,0,354,3.91,67,0.096,0.119,0.099,0.063,0.192,0.017,0.020,0.070,0.170,0.153,0.258,0.258,0.235,0.249,12,0
1,1,354,3.91,67,0.166,0.068,0.090,0.151,0.090,0.117,0.140,0.094,0.046,0.037,0.517,0.073,0.408,0.002,12,0
2,2,354,3.91,67,0.118,0.179,0.031,0.057,0.045,0.219,0.063,0.215,0.006,0.067,0.103,0.192,0.374,0.331,12,0
3,3,354,3.91,67,0.016,0.039,0.169,0.140,0.141,0.071,0.117,0.041,0.121,0.145,0.344,0.372,0.066,0.219,12,0
4,4,354,3.91,67,0.048,0.196,0.157,0.111,0.113,0.051,0.103,0.162,0.047,0.012,0.051,0.364,0.463,0.122,12,0
5,5,354,3.91,67,0.173,0.026,0.064,0.001,0.003,0.142,0.161,0.027,0.160,0.245,0.300,0.175,0.086,0.439,12,0
6,6,354,3.91,67,0.119,0.117,0.138,0.142,0.047,0.122,0.020,0.073,0.141,0.081,0.371,0.059,0.380,0.190,12,0
7,7,354,3.91,67,0.068,0.057,0.024,0.086,0.155,0.091,0.198,0.131,0.099,0.090,0.168,0.361,0.176,0.295,12,0
8,8,354,3.91,67,0.055,0.165,0.198,0.021,0.078,0.060,0.082,0.173,0.028,0.139,0.246,0.484,0.077,0.193,12,0
9,9,354,3.91,67,0.154,0.029,0.003,0.115,0.171,0.186,0.136,0.004,0.137,0.065,0.345,0.050,0.283,0.323,12,0


In [9]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Caution,A1,154.725160,0.570796
1,Caution,A2,175.865028,0.570796
2,Caution,A3,164.578264,0.570796
3,Caution,A4,350.394651,0.570796
4,Caution,A5,345.142203,0.570796
5,Good,A6,320.922553,0.558092
6,Good,B1,331.010148,0.465873
7,Good,B2,306.276815,0.311858
8,Good,B3,312.501844,0.330593
9,Good,B4,244.465881,0.334494


In [10]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,316.596269,12.864979,0.369441,0.084036,1
3,Formulation 4,252.343655,11.140854,0.359479,0.035334,1
4,Formulation 5,323.625351,2.417202,0.432044,0.031808,1
5,Formulation 6,225.073171,5.758277,0.299149,0.038186,1
6,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,Formulation 8,236.222805,6.701295,0.300872,0.048132,1
8,Formulation 9,266.542761,3.875382,0.387059,0.012545,1
9,Formulation 10,294.384922,8.616639,0.388992,0.050478,1


In [11]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,0,354,3.91,67,0.096,0.119,0.099,0.063,0.192,0.017,...,0.235,0.249,12,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,1,354,3.91,67,0.166,0.068,0.090,0.151,0.090,0.117,...,0.408,0.002,12,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,2,354,3.91,67,0.118,0.179,0.031,0.057,0.045,0.219,...,0.374,0.331,12,0,Formulation 3,316.596269,12.864979,0.369441,0.084036,1
3,3,354,3.91,67,0.016,0.039,0.169,0.140,0.141,0.071,...,0.066,0.219,12,0,Formulation 4,252.343655,11.140854,0.359479,0.035334,1
4,4,354,3.91,67,0.048,0.196,0.157,0.111,0.113,0.051,...,0.463,0.122,12,0,Formulation 5,323.625351,2.417202,0.432044,0.031808,1
5,5,354,3.91,67,0.173,0.026,0.064,0.001,0.003,0.142,...,0.086,0.439,12,0,Formulation 6,225.073171,5.758277,0.299149,0.038186,1
6,6,354,3.91,67,0.119,0.117,0.138,0.142,0.047,0.122,...,0.380,0.190,12,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,7,354,3.91,67,0.068,0.057,0.024,0.086,0.155,0.091,...,0.176,0.295,12,0,Formulation 8,236.222805,6.701295,0.300872,0.048132,1
8,8,354,3.91,67,0.055,0.165,0.198,0.021,0.078,0.060,...,0.077,0.193,12,0,Formulation 9,266.542761,3.875382,0.387059,0.012545,1
9,9,354,3.91,67,0.154,0.029,0.003,0.115,0.171,0.186,...,0.283,0.323,12,0,Formulation 10,294.384922,8.616639,0.388992,0.050478,1


In [12]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,0,354,3.91,67,0.096,0.119,0.099,0.063,0.192,0.017,...,12,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
1,1,354,3.91,67,0.166,0.068,0.090,0.151,0.090,0.117,...,12,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,2,354,3.91,67,0.118,0.179,0.031,0.057,0.045,0.219,...,12,0,Formulation 3,316.596269,12.864979,0.369441,0.084036,1,236.0,0
3,3,354,3.91,67,0.016,0.039,0.169,0.140,0.141,0.071,...,12,0,Formulation 4,252.343655,11.140854,0.359479,0.035334,1,32.0,0
4,4,354,3.91,67,0.048,0.196,0.157,0.111,0.113,0.051,...,12,0,Formulation 5,323.625351,2.417202,0.432044,0.031808,1,96.0,0
5,5,354,3.91,67,0.173,0.026,0.064,0.001,0.003,0.142,...,12,0,Formulation 6,225.073171,5.758277,0.299149,0.038186,1,346.0,0
6,6,354,3.91,67,0.119,0.117,0.138,0.142,0.047,0.122,...,12,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
7,7,354,3.91,67,0.068,0.057,0.024,0.086,0.155,0.091,...,12,0,Formulation 8,236.222805,6.701295,0.300872,0.048132,1,136.0,0
8,8,354,3.91,67,0.055,0.165,0.198,0.021,0.078,0.060,...,12,0,Formulation 9,266.542761,3.875382,0.387059,0.012545,1,110.0,0
9,9,354,3.91,67,0.154,0.029,0.003,0.115,0.171,0.186,...,12,0,Formulation 10,294.384922,8.616639,0.388992,0.050478,1,308.0,0


In [13]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.0
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.0
2,1,0.316596,0.012865,0.118,0.0,0.369441,0.084036,1.0
3,1,0.252344,0.011141,0.016,0.0,0.359479,0.035334,1.0
4,1,0.323625,0.002417,0.048,0.0,0.432044,0.031808,1.0
5,1,0.225073,0.005758,0.173,0.0,0.299149,0.038186,1.0
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.0
7,1,0.236223,0.006701,0.068,0.0,0.300872,0.048132,1.0
8,1,0.266543,0.003875,0.055,0.0,0.387059,0.012545,1.0
9,1,0.294385,0.008617,0.154,0.0,0.388992,0.050478,1.0


In [14]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [15]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [16]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,0,354,3.91,67,0.096,0.119,0.099,0.063,0.192,0.017,...,1,0,Formulation 1,1.000000,0.000000,1.000000,0.000000,0,0.000,0
1,1,354,3.91,67,0.166,0.068,0.090,0.151,0.090,0.117,...,1,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,2,354,3.91,67,0.118,0.179,0.031,0.057,0.045,0.219,...,1,0,Formulation 3,0.316596,0.012865,0.369441,0.084036,1,0.118,0
3,3,354,3.91,67,0.016,0.039,0.169,0.140,0.141,0.071,...,1,0,Formulation 4,0.252344,0.011141,0.359479,0.035334,1,0.016,0
4,4,354,3.91,67,0.048,0.196,0.157,0.111,0.113,0.051,...,1,0,Formulation 5,0.323625,0.002417,0.432044,0.031808,1,0.048,0


In [17]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-11 09:00:52] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [18]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [19]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-11 09:00:54] ax.service.ax_client: Completed trial 0 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (1, 0)}.
[INFO 02-11 09:00:54] ax.service.ax_client: Completed trial 1 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (1, 0)}.
[INFO 02-11 09:00:54] ax.service.ax_client: Completed trial 2 with data: {'Solu': (0.118, 0), 'Size': (0.316596, 0.012865), 'PDI': (0.369441, 0.084036), 'Complexity': (1, 0)}.
[INFO 02-11 09:00:54] ax.service.ax_client: Completed trial 3 with data: {'Solu': (0.016, 0), 'Size': (0.252344, 0.011141), 'PDI': (0.359479, 0.035334), 'Complexity': (1, 0)}.
[INFO 02-11 09:00:54] ax.service.ax_client: Completed trial 4 with data: {'Solu': (0.048, 0), 'Size': (0.323625, 0.002417), 'PDI': (0.432044, 0.031808), 'Complexity': (1, 0)}.
[INFO 02-11 09:00:54] ax.service.ax_client: Completed trial 5 with data: {'Solu': (0.173, 0), 'Size': (0.225073, 0.005758), 'PDI': (0.299149, 0.038186), 'Complexity